# Multi-Layer Perceptron

## Import Libraries

In [1]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
X, y = make_classification(n_samples=1000, n_features=20, n_informative=10, n_redundant=5, n_classes=2, random_state=42)

In [3]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [4]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

## Model

In [5]:
import numpy as np
from sklearn.model_selection import KFold
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, Subset

device = "cuda" if torch.cuda.is_available() else\
         "mps" if torch.backends.mps.is_available() else\
         "cpu"

In [6]:
class MLP(nn.Module):
    def __init__(self, input_dim=20, hidden_dim=16, output_dim=1):
        super(MLP, self).__init__()

        self.fc = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.fc(x)

In [7]:
def train(X, y):
    n_features = X.shape[1]
    lr         = 1e-3
    batch_size = 32
    num_epochs = 100
    steps      = num_epochs // 10
    k          = 5

    X_torch = torch.tensor(X, dtype=torch.float32)
    y_torch = torch.tensor(y, dtype=torch.float32).unsqueeze(1)
    dataset = TensorDataset(X_torch, y_torch)

    kfold = KFold(n_splits=k, shuffle=True, random_state=42)
    fold_accuracies = []

    for fold, (train_index, val_index) in enumerate(kfold.split(dataset)):
        print(f"\n--- Fold {fold+1}/{k} ---")
        train_sub = Subset(dataset, train_index)
        val_sub   = Subset(dataset, val_index)

        train_loader = DataLoader(train_sub, batch_size=batch_size, shuffle=True)
        val_loader   = DataLoader(val_sub, batch_size=batch_size, shuffle=False)
    
        model = MLP(input_dim=n_features)
        model.to(device)
        optimizer = optim.Adam(model.parameters(), lr=lr)
        # criterion = nn.BCEWithLogitsLoss()
        criterion = nn.BCELoss()

        for epoch in range(num_epochs):
            model.train()
            total_loss = 0
            for x_batch, y_batch in train_loader:
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)

                optimizer.zero_grad()
                logits  = model(x_batch)
                loss    = criterion(logits, y_batch.float())
                total_loss += loss.item()
                loss.backward()
                optimizer.step()
    
            if (epoch + 1) % steps == 0:
                avg_loss = total_loss / len(train_loader)
                print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {avg_loss:.4f}")

        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for x_batch, y_batch in val_loader:
                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)
                logits = model(x_batch)
                preds  = (logits >= 0.5).long()
                correct += (preds == y_batch).sum().item()
                total += y_batch.size(0)

        acc = correct / total
        fold_accuracies.append(acc)
        print(f"Fold {fold+1} Accuracy: {acc:.2f}")

    print(f"\nAverage accuracy across {k} folds: {np.mean(fold_accuracies):.2f}")
    return model

In [8]:
if __name__ == '__main__':
    model = train(X_train, y_train)

    batch_size = 32
    X_torch = torch.tensor(X_test, dtype=torch.float32)
    y_torch = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)
    dataset = TensorDataset(X_torch, y_torch)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    accuracy = []
    model.eval()
    with torch.no_grad():
        total, correct = 0, 0
        for x_batch, y_batch in loader:
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            
            logits  = model(x_batch)
            preds   = (logits >= 0.5).long()
            correct += (preds == y_batch).sum().item()
            total += y_batch.size(0)

        acc = correct / total
        accuracy.append(acc)

    print(f"Test Accuracy: {np.mean(accuracy)*100:.2f}")


--- Fold 1/5 ---
Epoch 10/100, Loss: 0.4236
Epoch 20/100, Loss: 0.2696
Epoch 30/100, Loss: 0.2098
Epoch 40/100, Loss: 0.1747
Epoch 50/100, Loss: 0.1497
Epoch 60/100, Loss: 0.1312
Epoch 70/100, Loss: 0.1165
Epoch 80/100, Loss: 0.1042
Epoch 90/100, Loss: 0.0935
Epoch 100/100, Loss: 0.0846
Fold 1 Accuracy: 0.96

--- Fold 2/5 ---
Epoch 10/100, Loss: 0.4268
Epoch 20/100, Loss: 0.2856
Epoch 30/100, Loss: 0.2242
Epoch 40/100, Loss: 0.1867
Epoch 50/100, Loss: 0.1607
Epoch 60/100, Loss: 0.1408
Epoch 70/100, Loss: 0.1243
Epoch 80/100, Loss: 0.1111
Epoch 90/100, Loss: 0.1008
Epoch 100/100, Loss: 0.0917
Fold 2 Accuracy: 0.93

--- Fold 3/5 ---
Epoch 10/100, Loss: 0.4136
Epoch 20/100, Loss: 0.2694
Epoch 30/100, Loss: 0.2068
Epoch 40/100, Loss: 0.1695
Epoch 50/100, Loss: 0.1459
Epoch 60/100, Loss: 0.1277
Epoch 70/100, Loss: 0.1134
Epoch 80/100, Loss: 0.1011
Epoch 90/100, Loss: 0.0908
Epoch 100/100, Loss: 0.0822
Fold 3 Accuracy: 0.95

--- Fold 4/5 ---
Epoch 10/100, Loss: 0.4462
Epoch 20/100, Loss: 0.